In [1]:
# 3D Event Display for events in the ANNIE Tank

%matplotlib
import sys                      
import numpy as np
import uproot3 as uproot
import matplotlib.pyplot as plt
import pandas


# ------------------------------------------------------------------- #
file = uproot.open('MC_Data/muon_swarm/muon_swarm.ntuple.root')
# ------------------------------------------------------------------- #


# cluster-level information
T = file['phaseIITankClusterTree']

clustereventNumber = T['eventNumber'].array()
clusterNumber = T['clusterNumber'].array()
clusterPE = T['clusterPE'].array()
clusterMaxPE = T['clusterMaxPE'].array()
clusterChargeBalance = T['clusterChargeBalance'].array()
clusterHits = T['clusterHits'].array()
clusterTime = T['clusterTime'].array()

# Total number of clusters (could be none, one, or multiple clusters per event)
N_clusters = len(clustereventNumber)

# hits-level information
Channel = T['hitChankeyMC'].array()
hitT = T['hitT'].array()
hitX = T['hitX'].array()
hitY = T['hitY'].array()
hitZ = T['hitZ'].array()
hitPE = T['hitPE'].array()


# MC Truth information (need to sort and assign them)
MC = file['phaseIITriggerTree']

eventNumber = MC['eventNumber'].array()  # Event Number
vtX = MC['trueVtxX'].array()             # {vertex information   
vtY = MC['trueVtxY'].array()             # ..
vtZ = MC['trueVtxZ'].array()             # }
vtT = MC['trueVtxTime'].array()          # "vertex time" i.e. initial time
dirX = MC['trueDirX'].array()            # {direction vectors of primary particle
dirY = MC['trueDirY'].array()            # ..
dirZ = MC['trueDirZ'].array()            # }
Energy = MC['trueMuonEnergy'].array()    # initial energy of the primary particle
Track_Length = MC['trueTrackLengthInWater'].array()  # track length of the primary particle in water 
                                                     # (distance from start point to stop point or the
                                                     # distance from the start vertex to a tank wall (if the particle exited))
    
N_events = len(eventNumber)    # Number of WCSim generated events

# sort events that have an associated cluster event number
vtX = [vtX[int(en)]/100 for en in clustereventNumber]
vtY = [vtY[int(en)]/100 for en in clustereventNumber]
vtZ = [vtZ[int(en)]/100 for en in clustereventNumber]
# divide by 100     ^  to convert vertex units [cm] into [m]
vtT = [vtT[int(en)] for en in clustereventNumber]
dirX = [dirX[int(en)] for en in clustereventNumber]
dirY = [dirY[int(en)] for en in clustereventNumber]
dirZ = [dirZ[int(en)] for en in clustereventNumber]
Energy = [Energy[int(en)] for en in clustereventNumber]
Track_Length = [Track_Length[int(en)] for en in clustereventNumber]

# record how many clusters per WCSim event
clusters_per_event = [(clustereventNumber.tolist()).count(i) for i in eventNumber]
cluster_count = [clusters_per_event[int(en)] for en in clustereventNumber]

# # # # # # # # # # # # # # # # # # # # # # # # # # #
# MC Truth Vertex and direction information
origin = np.zeros([N_clusters,3])
dir_vector = np.zeros([N_clusters,3])
for i in range(N_clusters):
    origin[i][0] = vtZ[i]; dir_vector[i][0] = dirZ[i]
    origin[i][1] = vtX[i]; dir_vector[i][1] = dirX[i]
    origin[i][2] = vtY[i]; dir_vector[i][2] = dirY[i]

    
# # # # # # # # # # # # # # # # # # # # # # # # # # #
### Load in and Construct the Detector Geometry ###

# Read Geometry.csv file to get PMT location info
df = pandas.read_csv('FullTankPMTGeometry.csv')

channel_number = []; location = []; panel_number = []
x_position = []; y_position = []; z_position = []

# The LoadGeometry File does not share the same origin point as the WCSim output after ToolAnalysis.

# vertical center (y) is at a height of y = -14.46 cm
# x-axis is fine
# z-axis (beamline) is offset by 1.681 m
# tank center is therefore at (0,-14.46, 168.1) [cm]

for i in range(len(df['channel_num'])):   # loop over PMTs
    channel_number.append(df['channel_num'][i])
    location.append(df['detector_tank_location'][i])
    x_position.append(df['x_pos'][i]+0)
    y_position.append(df['y_pos'][i]+0.1446)
    z_position.append(df['z_pos'][i]-1.681)
    panel_number.append(df['panel_number'][i])

# Find the dimensions of the detector (radius, height, etc..)
height_detector = max(y_position) - min(y_position)
radius_detector = (max(z_position) - min(z_position))/2   

# # # # # # # # # # # # # # # # # # # # # # # # # # # #

print('Data Loaded\n')
print('#################')
print('\nNumber of Events = ', N_events)
print('Number of Clusters = ', N_clusters)

Using matplotlib backend: <object object at 0x7fb689c6c9d0>
Data Loaded

#################

Number of Events =  750
Number of Clusters =  1152


In [12]:
%matplotlib

primary_particle = r'$\mu^-$'          # MC particle type

How_many_clusters = 1                  # How many events should be displayed/saved?                                         

cluster_offset = 2                     # Adjust to plot a later cluster 
                                       # (this will be the first event/cluster)

Name = '3D_Plots/MC muon ANNIEEvent'   # Name of the Plot/Event?

save_pics = True                       # Save .png images of the event display?

plot_other_PMTs = False                # plot the non-hit PMTs?
                                       # on = good for visualizing the tank geometry
                                       # off = good for visualizing the event and the PMTs that were hit

PMT_markersize = 100                   # default is 100

display_events = False                  # open the plot in an external window to view it?
                                       # (setting = False will just save the image, but not display it)


###################################################################

for i in range(cluster_offset, How_many_clusters + cluster_offset):

    print('\nEvent ' + str(int(clustereventNumber[i])) + \
          ' | Cluster ' + str(int(clusterNumber[i]+1)) + '/' + str(cluster_count[i]))
    
    
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    ############################ Charge Plot ##############################
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

    fig = plt.figure(figsize = (15,10))
    
    ax = fig.add_subplot(projection='3d', computed_zorder=False)   # zorder was not manual as of 3.5.0
    
    fig.patch.set_facecolor('xkcd:black')

    # Plot PMTs that were not hit  (assigned black)
    if plot_other_PMTs == True:
        for cn in range(len(channel_number)):
            if channel_number[cn] not in unique_PMTs[i][0]:   # construct base geometry of PMTs not hit
                ax.scatter(z_position[cn], x_position[cn], y_position[cn], s = PMT_markersize, color = 'black', zorder = 5)
                
    p = ax.scatter(hitZ[i], hitX[i], hitY[i], c = hitPE[i], cmap=plt.cm.plasma,
                s = PMT_markersize, vmin = 0, vmax = clusterMaxPE[i], alpha = 1, zorder = 5)

    cb = plt.colorbar(p)   # colorbar depecting the amount of charge recorded on each PMT
    cb.set_label(label = 'Photoelectrons', color = 'white')
    cb.ax.yaxis.set_tick_params(color='white')
    cb.outline.set_edgecolor('white')
    plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='white')

    # # # # # # # # # # # # # # # # #

    # Statistics
    hits_text = str(int(clusterHits[i])) + ' hits / ' + str(round(clusterPE[i],2)) + ' p.e.'
    ax.text2D(0.8,0.93,hits_text,size = 12,transform = ax.transAxes, color = 'white')
    chargebalance = r'$Q_{b}$' + ' = ' + str(round(clusterChargeBalance[i],3)) + ' ; ' + r'$PE_{max}$' + ' = ' + str(round(clusterMaxPE[i],2))
    ax.text2D(0.75,0.88,chargebalance,size = 12,transform = ax.transAxes, color = 'white')
    
    # Cluster Number info
    cluster_text = 'Cluster ' + str(int(clusterNumber[i]+1)) + '/' + str(cluster_count[i])
    ax.text2D(0.05,0.88,cluster_text,size = 12,transform = ax.transAxes, color = 'white')
    
    # # # # # # # # # # # # # # # # #
    
    # Plot vertex truth position - figure is (z,x,y) but loaded origin is (x,y,z)
    ax.scatter(origin[i][0],origin[i][1],origin[i][2], 
               s = PMT_markersize, color = 'red', marker = '*', zorder = 1, label = 'Truth Vertex')
    
    # Plot the truth direction (momentum)
    scale = Track_Length[i]   # Scale of arrow will be the real track length of the primary gamma
    ax.quiver(origin[i][0], origin[i][1], origin[i][2], dirZ[i]*scale, dirX[i]*scale, dirY[i]*scale, 
            color = 'red', label = 'Primary ' + primary_particle + ' initial momentum and final track length', zorder = 1)
    
    # Important to note that the track length is simply the difference between the start and stop vertex of the primary particle. 
    # The initial direction is truth, but the actual direction as the primary particle propagates may change. 
    # Therefore, the quiver within the plot is more of a visual representation of the propagation length of the initial particle.

    ax.set_aspect('auto')
    ax.set_xlabel('z [m]', color = 'white')
    ax.set_ylabel('x [m]', color = 'white')
    ax.set_zlabel('y [m]', color = 'white')
    ax.xaxis.set_tick_params(color='white')
    ax.yaxis.set_tick_params(color='white')
    ax.zaxis.set_tick_params(color='white')
    plt.setp(plt.getp(ax.axes, 'xticklabels'), color='white')
    plt.setp(plt.getp(ax.axes, 'yticklabels'), color='white')
    plt.setp(plt.getp(ax.axes, 'zticklabels'), color='white')

    ax.set_facecolor('xkcd:black')

    # Create cubic bounding box to simulate equal aspect ratio
    max_range = np.array([max(z_position)-min(z_position), max(x_position)-min(x_position), max(y_position)-min(y_position)]).max()
    Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(max(z_position)+min(z_position))
    Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(max(x_position)+min(x_position))
    Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(max(y_position)+min(y_position))
    for xb, yb, zb in zip(Xb, Yb, Zb):
        ax.plot([xb], [yb], [zb], 'w')

    plt.title('ANNIE Tank 3D Event ' + str(int(clustereventNumber[i])) + ' | Charge Plot', color = 'white')
    plt.legend(loc = 'upper left')
    
    path = Name + ' ' + str(int(clustereventNumber[i])) + ' Cluster ' + str(int(clusterNumber[i]+1)) + ' Charge Plot.png'
    if save_pics == True:
        plt.savefig(path,dpi=300, bbox_inches='tight', pad_inches=.3,facecolor = 'black')
    if display_events == True:
        plt.show()
    else:
        plt.close()


    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    ############################ Timing Plot ##############################
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    
    fig = plt.figure(figsize = (15,10))
    
    ax = fig.add_subplot(projection='3d', computed_zorder=False)   # zorder was not manual as of 3.5.0
    
    fig.patch.set_facecolor('xkcd:black')

    # Plot PMTs that were not hit  (assigned black)
    if plot_other_PMTs == True:
        for cn in range(len(channel_number)):
            if channel_number[cn] not in unique_PMTs[i][0]:   # construct base geometry of PMTs not hit
                ax.scatter(z_position[cn], x_position[cn], y_position[cn], s = PMT_markersize, color = 'black', zorder = 5)
    
    # chain the timing to the first PMT hit
    timing = [(hitT[i][j]) - min(hitT[i]) for j in range(len(hitT[i]))]

    p = ax.scatter(hitZ[i], hitX[i], hitY[i], c = timing, cmap=plt.cm.cool, vmin = min(timing), vmax = max(timing), 
                   s = PMT_markersize, alpha = 1, zorder = 5)

    cb = plt.colorbar(p)   # colorbar depecting the amount of charge recorded on each PMT
    cb.set_label(label = 'Time [nanoseconds]', color = 'white')
    cb.ax.yaxis.set_tick_params(color='white')
    cb.outline.set_edgecolor('white')
    plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='white')
    
    # # # # # # # # # # # # # # # # #

    # Statistics
    hits_text = str(int(clusterHits[i])) + ' hits / ' + str(round(clusterPE[i],2)) + ' p.e.'
    ax.text2D(0.78,0.93,hits_text,size = 12,transform = ax.transAxes, color = 'white')
    
    import statistics as st
    t_text = r'$(t_{first},t_{median},t_{last})$' + ' = (' + str(int(min(timing))) + ',' + str(int(st.median(timing))) + ',' + str(int(max(timing))) + ') ns'
    ax.text2D(0.72,0.88,t_text,size = 12,transform = ax.transAxes, color = 'white')
    
    # Cluster Number info
    cluster_text = 'Cluster ' + str(int(clusterNumber[i]+1)) + '/' + str(cluster_count[i])
    ax.text2D(0.05,0.88,cluster_text,size = 12,transform = ax.transAxes, color = 'white')
    
    # # # # # # # # # # # # # # # # #

    # Plot vertex truth position - figure is (z,x,y) but loaded origin is (x,y,z)
    ax.scatter(origin[i][0],origin[i][1],origin[i][2], s = PMT_markersize, color = 'red', marker = '*', zorder = 1, label = 'Truth Vertex')
    
    # Plot the truth direction (momentum)
    scale = Track_Length[i]   # Scale of arrow will be the real track length of the primary gamma
    ax.quiver(origin[i][0], origin[i][1], origin[i][2], dirZ[i]*scale, dirX[i]*scale, dirY[i]*scale, 
            color = 'red', label = 'Primary ' + primary_particle + ' initial momentum and final track length', zorder = 1)

    ax.set_aspect('auto')
    ax.set_xlabel('z [m]', color = 'white')
    ax.set_ylabel('x [m]', color = 'white')
    ax.set_zlabel('y [m]', color = 'white')
    ax.xaxis.set_tick_params(color='white')
    ax.yaxis.set_tick_params(color='white')
    ax.zaxis.set_tick_params(color='white')
    plt.setp(plt.getp(ax.axes, 'xticklabels'), color='white')
    plt.setp(plt.getp(ax.axes, 'yticklabels'), color='white')
    plt.setp(plt.getp(ax.axes, 'zticklabels'), color='white')

    ax.set_facecolor('xkcd:black')

    # Create cubic bounding box to simulate equal aspect ratio
    max_range = np.array([max(z_position)-min(z_position), max(x_position)-min(x_position), max(y_position)-min(y_position)]).max()
    Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(max(z_position)+min(z_position))
    Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(max(x_position)+min(x_position))
    Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(max(y_position)+min(y_position))
    for xb, yb, zb in zip(Xb, Yb, Zb):
        ax.plot([xb], [yb], [zb], 'w')

    plt.title('ANNIE Tank 3D Event ' + str(int(clustereventNumber[i])) + ' | Timing Plot', color = 'white')
    plt.legend(shadow=True, loc = 'upper left')

    path = Name + ' ' + str(int(clustereventNumber[i])) + ' Cluster ' + str(int(clusterNumber[i]+1)) + ' Timing Plot.png'
    if save_pics == True:
        plt.savefig(path,dpi=300, bbox_inches='tight', pad_inches=.3,facecolor = 'black')
    if display_events == True:
        plt.show()
    else:
        plt.close()
        
        
################################################

print('\ndone')

Using matplotlib backend: MacOSX

Event 1 | Cluster 1/2

done
